In [4]:
%pip install --upgrade pip numpy requests unidecode pandas

Note: you may need to restart the kernel to use updated packages.


In [5]:
# somarec_pipeline.py
# -*- coding: utf-8 -*-
import os, re, json, time, hashlib, pathlib, logging
from typing import List, Dict, Optional
import pandas as pd
import numpy as np
import requests
from unidecode import unidecode

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s %(message)s"
)
log = logging.getLogger("somarec")

In [6]:
# -------------------- config --------------------
DATA = pathlib.Path("somarec_data")
RAW = DATA / "raw"
DERIVED = DATA / "derived"
CACHE = DATA / "cache"
for p in (RAW, DERIVED, CACHE): p.mkdir(parents=True, exist_ok=True)

USER_AGENT = "SomaRec/0.8 (nicole.nasengo@example.ke)"  # set to a reachable address
HEADERS_JSON = {"User-Agent": USER_AGENT, "Accept": "application/json"}

# Optional: load .env and keys
try:
    from dotenv import load_dotenv; load_dotenv()
except Exception:
    pass
# Use environment variable for Google Books API key
try:
    from google.colab import userdata
    GB_KEY = userdata.get("GOOGLE_BOOKS_API_KEY")
except Exception as e:
    log.warning(f"Could not load Google Books API key from Colab secrets: {e}")
    GB_KEY = None


2025-11-07 13:09:35,677 WARNING Could not load Google Books API key from Colab secrets: No module named 'google'


In [7]:
# -------------------- utils --------------------
def norm_space(s: str) -> str:
    return re.sub(r"\s+", " ", (s or "").strip())

def norm_key(s: str) -> str:
    s = unidecode((s or "").lower())
    s = re.sub(r"[\.,'’`\"\-]+", " ", s)
    return re.sub(r"\s+", " ", s).strip()

def cache_read(key: str) -> Optional[dict]:
    p = CACHE / (hashlib.md5(key.encode()).hexdigest() + ".json")
    if p.exists():
        try: return json.loads(p.read_text())
        except Exception: return None
    return None

def cache_write(key: str, val: dict) -> None:
    p = CACHE / (hashlib.md5(key.encode()).hexdigest() + ".json")
    p.write_text(json.dumps(val, ensure_ascii=False), encoding="utf-8", errors="replace")

def safe_get(df: pd.DataFrame, col: str) -> pd.Series:
    return df[col] if col in df.columns else pd.Series([None]*len(df))


In [8]:

# -------------------- 1) data ingest --------------------
def load_kaggle_bookcrossing(
    bx_books_csv: Optional[str] = None,
    bx_ratings_csv: Optional[str] = None
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Load Book-Crossing Books and Ratings. Provide explicit paths or place files under RAW.
    """
    # Prefer explicit paths; else try common names
    books_path = bx_books_csv or next((p for p in RAW.glob("**/BX-Books.csv")), None) \
                 or next((p for p in RAW.glob("**/Books.csv")), None)
    ratings_path = bx_ratings_csv or next((p for p in RAW.glob("**/BX-Book-Ratings.csv")), None)

    if not books_path:
        log.warning("Book-Crossing books file not found under somarec_data/raw.")
        books = pd.DataFrame()
    else:
        # BX is usually latin-1 with commas
        books = pd.read_csv(books_path, encoding="latin-1", on_bad_lines="skip")

    if ratings_path:
        ratings = pd.read_csv(ratings_path, encoding="latin-1", on_bad_lines="skip")
    else:
        ratings = pd.DataFrame()

    log.info(f"BX Books shape: {books.shape}; Ratings shape: {ratings.shape}")
    return books, ratings


In [9]:

# -------------------- 2) wikidata authors + works --------------------
WD_ENDPOINT = "https://query.wikidata.org/sparql"

def wd_sparql(q: str) -> dict:
    r = requests.get(
        WD_ENDPOINT,
        params={"query": q, "format": "json"},
        headers={"User-Agent": USER_AGENT, "Accept": "application/sparql-results+json"},
        timeout=60
    )
    r.raise_for_status()
    time.sleep(0.5)  # be nice to WDQS
    return r.json()

SPARQL_AUTHORS_KE = r"""
SELECT DISTINCT ?author ?authorLabel (GROUP_CONCAT(DISTINCT ?alias; separator="\n") AS ?aliases)
WHERE {
  ?author wdt:P31 wd:Q5 .
  ?author wdt:P27 wd:Q114 .    # human, Kenyan citizenship
  ?author wdt:P106 ?occ .
  VALUES ?occ { wd:Q36180 wd:Q49757 wd:Q482980 wd:Q214917 wd:Q6625963 wd:Q18844224 wd:Q117291434 }
  OPTIONAL { ?author skos:altLabel ?alias FILTER (LANG(?alias)="en") }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en,sw,en-gb". }
}
GROUP BY ?author ?authorLabel
ORDER BY ?authorLabel
"""

SPARQL_WORKS_KE = r"""
SELECT DISTINCT ?work ?workLabel ?authorLabel ?pubdate ?langLabel ?publisherLabel ?isbn13 ?isbn10 ?ol
WHERE {
  ?author wdt:P31 wd:Q5 ; wdt:P27 wd:Q114 .
  ?work wdt:P50 ?author .
  VALUES ?litclass {
    wd:Q47461344  # written work
    wd:Q8261      # novel
    wd:Q7725634   # literary work
    wd:Q25379     # play
    wd:Q7726210   # short story
    wd:Q49848     # poem
  }
  ?work wdt:P31 ?litclass .
  OPTIONAL { ?work wdt:P577 ?pubdate. }
  OPTIONAL { ?work wdt:P407 ?lang. }
  OPTIONAL { ?work wdt:P123 ?publisher. }
  OPTIONAL { ?work wdt:P212 ?isbn13. }
  OPTIONAL { ?work wdt:P957 ?isbn10. }
  OPTIONAL { ?work wdt:P648 ?ol. }  # Open Library work id
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en,sw,en-gb". }
}
"""

def get_kenyan_authors() -> pd.DataFrame:
    js = wd_sparql(SPARQL_AUTHORS_KE)
    rows = js["results"]["bindings"]
    out = []
    for r in rows:
        name = r["authorLabel"]["value"].strip()
        aliases = (r.get("aliases", {}).get("value") or "")
        alias_set = sorted(set([a.strip() for a in aliases.split("\n") if a.strip()] + [name]))
        out.append({"name": name, "aliases": "\n".join(alias_set)})
    df = pd.DataFrame(out).sort_values("name").reset_index(drop=True)
    df.to_csv(DERIVED / "kenyan_authors.csv", index=False)
    log.info(f"Kenyan authors: {df.shape}")
    return df

def get_wikidata_works() -> pd.DataFrame:
    js = wd_sparql(SPARQL_WORKS_KE)
    rows = js["results"]["bindings"]
    def v(r, k): return r.get(k, {}).get("value")
    wk = [{
        "wd_work": v(r,"work"),
        "title": v(r,"workLabel"),
        "author": v(r,"authorLabel"),
        "pubdate": v(r,"pubdate"),
        "language": v(r,"langLabel"),
        "publisher": v(r,"publisherLabel"),
        "isbn13": v(r,"isbn13"),
        "isbn10": v(r,"isbn10"),
        "ol_work_id": v(r,"ol"),
        "source": "wikidata"
    } for r in rows]
    df = pd.DataFrame(wk).drop_duplicates()
    log.info(f"Wikidata works: {df.shape}")
    return df


In [10]:
# -------------------- 3) open library harvest --------------------
OL_SEARCH = "https://openlibrary.org/search.json"

def ol_search(params: dict, delay: float = 0.25) -> dict:
    key = "ol::" + json.dumps(params, sort_keys=True)
    hit = cache_read(key)
    if hit: return hit
    r = requests.get(OL_SEARCH, params=params, headers=HEADERS_JSON, timeout=45)
    r.raise_for_status()
    js = r.json()
    cache_write(key, js)
    time.sleep(delay)
    return js

KE_PUBLISHERS = [
    "East African Educational Publishers",
    "Kenya Literature Bureau",
    "Longhorn Publishers",
    "Jomo Kenyatta Foundation",
]

def build_alias_set(authors_df: pd.DataFrame) -> set[str]:
    aliases = set()
    for s in authors_df["aliases"]:
        for a in str(s).split("\n"):
            a = a.strip()
            if a:
                aliases.add(a)
    return { norm_key(a) for a in aliases }

def ol_collect_docs(seed_authors: list[str]) -> pd.DataFrame:
    docs = []
    # (1) author
    for a in seed_authors:
        js = ol_search({"author": a, "limit": 20})
        for d in js.get("docs", []):
            d["_source"] = "ol_author"; d["_author_q"] = a
            docs.append(d)
    # (2) publisher
    for pub in KE_PUBLISHERS:
        js = ol_search({"q": f'publisher:"{pub}"', "limit": 50})
        for d in js.get("docs", []):
            d["_source"] = "ol_publisher"; d["_publisher_q"] = pub
            docs.append(d)
    # (3) subject slice – Kenya, fiction
    try:
        sj = requests.get(
            "https://openlibrary.org/subjects/kenya_fiction.json?details=false&limit=200",
            headers=HEADERS_JSON, timeout=45
        ).json()
        time.sleep(0.25)
        for w in sj.get("works", []):
            docs.append({
                "title": w.get("title"),
                "author_name": [a.get("name") for a in (w.get("authors") or []) if a],
                "key": w.get("key"),
                "edition_count": w.get("edition_count"),
                "cover_i": (w.get("cover_id") or None),
                "_source": "ol_subject_kenya_fiction"
            })
    except Exception as e:
        log.warning(f"Open Library subject fetch failed: {e}")

    # Fetch descriptions for each doc using the Open Library API (details=true) if missing
    for doc in docs:
        if doc.get('key') and not doc.get('description'):
            try:
                details_url = f"https://openlibrary.org{doc['key']}.json"
                # Check cache first
                details_key = "ol_details::" + doc['key']
                details_js = cache_read(details_key)
                if not details_js:
                    details_r = requests.get(details_url, headers=HEADERS_JSON, timeout=45)
                    details_r.raise_for_status()
                    details_js = details_r.json()
                    cache_write(details_key, details_js)
                    time.sleep(0.25) # Be nice to OL

                desc = details_js.get('description')
                if desc:
                    # Description can be a string or a dict with a value key
                    if isinstance(desc, dict) and 'value' in desc:
                        doc['description'] = desc['value']
                    elif isinstance(desc, str):
                        doc['description'] = desc
            except Exception as e:
                log.warning(f"Failed to fetch OL description for {doc.get('key')}: {e}")

    return pd.DataFrame(docs)

In [11]:
# -------------------- 4) harmonize + dedupe --------------------
def map_to_canonical(df: pd.DataFrame) -> pd.DataFrame:
    """Map mixed columns to a consistent schema."""
    out = pd.DataFrame({
        "title": df.get("title", df.get("Book-Title")),
        "author": df.get("author", df.get("Book-Author")),
        "publisher": df.get("publisher"),
        "isbn10": df.get("isbn10", df.get("ISBN", None)),
        "isbn13": df.get("isbn13"),
        "language": df.get("language"),
        "pubdate": df.get("pubdate"),
        "ol_work_key": df.get("ol_work_id", df.get("key")),
        "cover_i": df.get("cover_i"),
        "image_url": df.get("image_url"),  # to be filled later
        "source": df.get("source"),
        "description": df.get("description") # Include description
    })
    return out

def dedupe_union(base: pd.DataFrame, add: pd.DataFrame) -> pd.DataFrame:
    base = map_to_canonical(base)
    add  = map_to_canonical(add)
    # Dedup key: prefer ISBN13 > ISBN10 > (title+author)
    base["key_isbn"] = base["isbn13"].fillna(base["isbn10"])
    add["key_isbn"]  = add["isbn13"].fillna(add["isbn10"])
    base["key_ta"]   = (base["title"].map(norm_key) + " | " + base["author"].map(norm_key))
    add["key_ta"]    = (add["title"].map(norm_key)  + " | "  + add["author"].map(norm_key))
    have = set(base["key_isbn"].dropna()) | set(base["key_ta"].dropna())
    is_new = ~(
        add["key_isbn"].isin(have) |
        add["key_ta"].isin(have)
    )
    # Drop columns that are all NA in either dataframe before concatenating,
    # but explicitly keep 'image_url' and 'description'
    cols_to_drop_base = set(base.columns[base.isnull().all()]) - {'image_url', 'description'}
    cols_to_drop_add = set(add.columns[add.isnull().all()]) - {'image_url', 'description'}

    merged = pd.concat([base.drop(columns=cols_to_drop_base, errors='ignore'),
                        add[is_new].drop(columns=cols_to_drop_add, errors='ignore')],
                       ignore_index=True)


    # Ensure key columns exist before dropping duplicates
    subset_cols = []
    if "key_isbn" in merged.columns:
        subset_cols.append("key_isbn")
    if "key_ta" in merged.columns:
        subset_cols.append("key_ta")

    if subset_cols:
        return merged.drop_duplicates(subset=subset_cols, keep="first")
    else:
        # If no key columns exist, return the merged dataframe as is (no deduplication)
        return merged

In [12]:
# -------------------- 5) Google Books enrichment (optional) --------------------
GB_BASE = "https://www.googleapis.com/books/v1"

def gb_search(q: str, max_results: int = 40, lang: Optional[str] = None) -> dict:
    params = {"q": q, "maxResults": min(max_results, 40)}
    if GB_KEY: params["key"] = GB_KEY
    if lang: params["langRestrict"] = lang
    key = "gb::" + json.dumps(params, sort_keys=True)
    hit = cache_read(key)
    if hit: return hit
    r = requests.get(f"{GB_BASE}/volumes", params=params,
                     headers={"User-Agent": USER_AGENT}, timeout=45)
    r.raise_for_status()
    js = r.json()
    cache_write(key, js)
    time.sleep(0.25)
    return js

def gb_flatten_item(item: dict, source: str) -> dict:
    v = item.get("volumeInfo", {}) or {}
    def extract_isbns():
        vals = []
        for ii in (v.get("industryIdentifiers") or []):
            ident = ii.get("identifier")
            if ident:
                vals.append(re.sub(r"[^\dxX]", "", ident))
        # Return both formats where possible
        isbn10 = next((x for x in vals if len(x) == 10), None)
        isbn13 = next((x for x in vals if len(x) == 13), None)
        return isbn10, isbn13
    i10, i13 = extract_isbns()
    return {
        "_src": source,
        "gb_id": item.get("id"),
        "title": v.get("title"),
        "authors": "\n".join(v.get("authors") or []),
        "publisher": v.get("publisher"),
        "publishedDate": v.get("publishedDate"),
        "categories": "\n".join(v.get("categories") or []),
        "avg_rating": v.get("averageRating"),
        "ratings_count": v.get("ratingsCount"),
        "thumbnail": (v.get("imageLinks") or {}).get("thumbnail"),
        "infoLink": v.get("infoLink"),
        "isbn10": i10, "isbn13": i13,
        "language": v.get("language"),
        "description": v.get("description") # Add description field
    }

In [ ]:
# -------------------- 6) main orchestrator --------------------
def run_pipeline():
    bx_books, bx_ratings = load_kaggle_bookcrossing()

    # Wikidata authors & works
    authors_df = get_kenyan_authors()
    wd_works   = get_wikidata_works()

    # Build exact + alias matches for Kaggle base (Book-Crossing books)
    alias_norm = {norm_key(a) for a in authors_df["aliases"].str.split("\n").sum()}
    def is_kenyan_author(name: str) -> bool:
        return norm_key(name) in alias_norm if isinstance(name, str) else False

    bx_books_kenya = pd.DataFrame()
    if not bx_books.empty:
        auth_col = next((c for c in bx_books.columns if norm_key(c) in ("book author","author","authors")), None)
        if auth_col:
            bx_books_kenya = bx_books[bx_books[auth_col].apply(is_kenyan_author)].copy()
            bx_books_kenya["source"] = "kaggle_bx"


    # Open Library harvest
    seed_authors = sorted({ norm_space(a) for a in authors_df["name"].tolist() })
    ol_docs = ol_collect_docs(seed_authors)

    # Normalize OL docs → canonical rows
    rows = []
    for _, d in ol_docs.iterrows():
        title = d.get("title") or d.get("title_suggest")
        authors = d.get("author_name") or []
        if not title or not authors: continue
        # Add a check here to ensure authors is a list before iterating
        if not isinstance(authors, list) or not any(norm_key(a) in alias_norm for a in authors): continue

        # Description is now fetched within ol_collect_docs
        description = d.get("description")


        rows.append({
            "title": title,
            "author": authors[0],
            "publisher": "\n".join((d.get("publisher") or [])[:5]) if d.get("publisher") else None,
            "isbn13": None, "isbn10": None,
            "language": None,
            "pubdate": d.get("first_publish_year"),
            "ol_work_key": d.get("key"),
            "cover_i": d.get("cover_i"),
            "image_url": f"https://covers.openlibrary.org/b/id/{d.get('cover_i')}-L.jpg" if d.get('cover_i') else None,
            "source": d.get("_source"),
            "description": description # Include description

        })
    ol_finds = pd.DataFrame(rows).drop_duplicates()

    # Union: Kaggle (Kenyan subset) + Wikidata works + OL finds
    base = pd.concat([bx_books_kenya, wd_works], ignore_index=True, sort=False)
    augmented = dedupe_union(base, ol_finds)

    # Explicitly ensure 'description' and 'image_url' columns exist before enrichment
    # This should ideally be handled by dedupe_union now, but keeping as a failsafe
    if 'description' not in augmented.columns:
        augmented['description'] = None
    if 'image_url' not in augmented.columns:
        augmented['image_url'] = None

    # Debugging: Print columns before Google Books enrichment
    log.info(f"Columns in augmented DataFrame before Google Books enrichment: {augmented.columns.tolist()}")


    # (Optional) Google Books enrichment for missing descriptions and images
    if GB_KEY:
        log.info("Enriching with Google Books...")

        def enrich_with_gb(row):
            # Use .get for safe access to original values
            original_description = row.get('description')
            original_image_url = row.get('image_url')

            needs_enrichment = pd.isna(original_description) or pd.isna(original_image_url)

            if needs_enrichment:
                query = f"{row.get('title', '')} {row.get('author', '')}" # Use .get for safe access
                try:
                    search_results = gb_search(query, max_results=1)
                    if search_results and search_results.get('items'):
                        flat_item = gb_flatten_item(search_results['items'][0], "google_books_enrichment")
                        # Update row values if enrichment found data and original is missing
                        if flat_item.get('description') and pd.isna(original_description):
                            row['description'] = flat_item['description']
                        if flat_item.get('thumbnail') and pd.isna(original_image_url):
                            row['image_url'] = flat_item['thumbnail']
                except Exception as e:
                    log.warning(f"Google Books search failed for '{query}': {e}")
                    # Continue with original row if search fails

            return row # Return the potentially updated row

        # Apply the enrichment function row-wise
        # Use augmented.index to preserve the original index after apply
        augmented = augmented.apply(enrich_with_gb, axis=1)


    # Image URL preference: OL cover if present and no image_url from GB
    # This part was already handled during OL and GB enrichment, but keep as a final check
    # Apply image_url logic after potential GB enrichment
    def image_url(row):
        # Use .get for safe access
        if pd.notnull(row.get("image_url")):
            return row["image_url"]
        elif pd.notnull(row.get("cover_i")):
            try: return f"https://covers.openlibrary.org/b/id/{int(row['cover_i'])}-L.jpg"
            except Exception: return None
        return None
    # Only apply if image_url column exists
    if 'image_url' in augmented.columns:
        augmented["image_url"] = augmented.apply(image_url, axis=1)


    out_path = DERIVED / "kenyan_works_augmented.csv"
    augmented.to_csv(out_path, index=False, encoding="utf-8", errors="replace")
    log.info(f"Wrote {out_path} with {augmented.shape[0]} rows.")

    # (Optional) keep ratings for future recommender
    if not bx_ratings.empty:
        bx_ratings.to_csv(DERIVED / "bx_ratings.csv", index=False, encoding="utf-8", errors="replace")


if __name__ == "__main__":
    run_pipeline()

2025-11-07 13:09:35,826 WARNING Book-Crossing books file not found under somarec_data/raw.
2025-11-07 13:09:35,835 INFO BX Books shape: (0, 0); Ratings shape: (0, 0)
2025-11-07 13:09:38,976 INFO Kenyan authors: (120, 2)
2025-11-07 13:09:41,358 INFO Wikidata works: (115, 10)


In [ ]:
# Save the updated DataFrame (even with the Google Books enrichment error)
out_path = DERIVED / "kenyan_works_augmented.csv"
if 'augmented' in globals() and isinstance(augmented, pd.DataFrame):
    augmented.to_csv(out_path, index=False, encoding="utf-8", errors="replace")
    log.info(f"Saved current state of data to {out_path}.")
else:
    log.error("DataFrame 'augmented' not found. Cannot save.")

In [ ]:
import pandas as pd

# Reload the DataFrame
out_path = DERIVED / "kenyan_works_augmented.csv"
try:
    augmented_works_df = pd.read_csv(out_path)
    log.info(f"Reloaded data from {out_path}. Shape: {augmented_works_df.shape}")
    display(augmented_works_df.head())
except FileNotFoundError:
    log.error(f"Error: File not found at {out_path}. Please ensure the pipeline ran successfully at least once.")
    augmented_works_df = pd.DataFrame() # Create an empty DataFrame to avoid further errors

In [ ]:
import pandas as pd

augmented_works_df = pd.read_csv('somarec_data/derived/kenyan_works_augmented.csv')
display(augmented_works_df.head())

In [ ]:
# 1. Explore the data
print("DataFrame Info:")
augmented_works_df.info()

print("\nDataFrame Shape:")
print(augmented_works_df.shape)

print("\nDataFrame Description:")
display(augmented_works_df.describe())

In [ ]:
# 2. Analyze the sources
print("\nSource Counts:")
display(augmented_works_df['source'].value_counts())

In [ ]:
# 3. Prepare for recommender system - Fetch Image URLs

if not GB_KEY:
    print("Google Books API key not found. Please set the GOOGLE_BOOKS_API_KEY in Colab secrets.")
    # Still try to populate Open Library images even without GB key
    augmented_works_df_copy = augmented_works_df.copy()
    def ol_image_url(row):
        if pd.notnull(row.get("cover_i")):
            try: return f"https://covers.openlibrary.org/b/id/{int(row['cover_i'])}-L.jpg"
            except Exception: return None
        return None
    augmented_works_df_copy["image_url"] = augmented_works_df_copy.apply(ol_image_url, axis=1)
    augmented_works_df = augmented_works_df_copy
    print("Attempted to fetch Open Library image URLs.")
    display(augmented_works_df.dropna(subset=['image_url']).head())

else:
    print("Fetching image URLs from Google Books (fallback) and Open Library...")
    # Create a copy to avoid SettingWithCopyWarning
    augmented_works_df_copy = augmented_works_df.copy()

    import urllib.parse

    def ol_image_url(row):
        if pd.notnull(row.get("cover_i")):
            try:
                return f"https://covers.openlibrary.org/b/id/{int(row['cover_i'])}-L.jpg"
            except Exception:
                return None
        return None

    def gb_image_url(title, author, key):
        query = f"intitle:{title} inauthor:{author}"
        encoded_query = urllib.parse.quote(query)
        url = f"https://www.googleapis.com/books/v1/volumes?q={encoded_query}&maxResults=1&key={key}"
        try:
            # Reuse the existing gb_search logic which includes caching and error handling
            search_results = gb_search(f"{title} {author}", max_results=1) # Use title and author directly for gb_search function
            if search_results and search_results.get('items'):
                flat_item = gb_flatten_item(search_results['items'][0], "google_books_enrichment")
                if flat_item.get('thumbnail'):
                    return flat_item['thumbnail']
        except Exception as e:
            log.warning(f"Google Books search failed for '{query}': {e}")
            return None
        return None


    # First, try to get Open Library cover images if cover_i is available
    augmented_works_df_copy["image_url"] = augmented_works_df_copy.apply(ol_image_url, axis=1)

    # Then, for rows that still don't have an image URL, try Google Books
    for index, row in augmented_works_df_copy.iterrows():
        if pd.isnull(row['image_url']): # Only try Google Books if image_url is still missing
            augmented_works_df_copy.at[index, 'image_url'] = gb_image_url(row['title'], row['author'], GB_KEY)


    augmented_works_df = augmented_works_df_copy # Update the original DataFrame variable
    print("Image URL fetching complete.")
    # Display rows that now have an image URL
    display(augmented_works_df.dropna(subset=['image_url']).head())

In [ ]:
# Add description fetching to the ol_search_title_author function
def ol_search_title_author(title: str, author: str, limit: int = 10, delay: float = 0.2):
    params = {"title": title, "author": author, "limit": limit}
    r = requests.get(OL_SEARCH_URL, params=params, headers=HEADERS, timeout=45)
    r.raise_for_status()
    time.sleep(delay)
    js = r.json() or {}
    docs = js.get("docs", [])

    # Fetch descriptions for each doc using the Open Library API (details=true)
    # We'll do this for the top few results to avoid excessive requests
    for doc in docs[:5]: # Limit description fetching to top 5 results
        if doc.get('key'):
            try:
                details_url = f"https://openlibrary.org{doc['key']}.json"
                details_r = requests.get(details_url, headers=HEADERS, timeout=45)
                details_r.raise_for_status()
                details_js = details_r.json()
                description = details_js.get('description')
                if description:
                    # Description can be a string or a dict with a value key
                    if isinstance(description, dict) and 'value' in description:
                        doc['description'] = description['value']
                    elif isinstance(description, str):
                        doc['description'] = description
            except Exception as e:
                log.warning(f"Failed to fetch OL description for {doc.get('key')}: {e}")

    return docs

# Modify map_ol_to_row to include description
def map_ol_to_row(d, source="ol_enrichment"):
    # fields may be absent; guard carefully
    title = d.get("title") or d.get("title_suggest")
    authors = d.get("author_name") or []
    publisher_list = d.get("publisher") or []
    year = d.get("first_publish_year")
    cover_i = d.get("cover_i")
    isbns = d.get("isbn") or []
    isbn13 = next((x for x in isbns if isinstance(x, str) and len(re.sub(r"\\D","", x))==13), None)
    isbn10 = next((x for x in isbns if isinstance(x, str) and len(re.sub(r"\\D","", x))==10), None)
    key = d.get("key")  # often /works/OL...W
    description = d.get("description") # Add description field

    row = {
        "title": title,
        "author": authors[0] if authors else None,
        "publisher": "\n".join(publisher_list[:5]) if publisher_list else None,
        "isbn13": isbn13, "isbn10": isbn10,
        "language": None,  # OL search not reliable here
        "pubdate": year,   # we'll cast to year later
        "ol_work_key": key,
        "cover_i": cover_i,
        "image_url": f"https://covers.openlibrary.org/b/id/{cover_i}-L.jpg" if cover_i else None,
        "source": source,
        "description": description # Include description in the row
    }
    return row

In [ ]:
print(f"Number of works with image URLs: {augmented_works_df['image_url'].notna().sum()}")
display(augmented_works_df.dropna(subset=['image_url']).head())

In [ ]:
# 2. Analyze the sources
print("\nSource Counts:")
display(augmented_works_df['source'].value_counts())

In [ ]:
# Explore the distribution of publication dates

# Drop rows with missing pubdates for this analysis
pubdates_present_df = augmented_works_df.dropna(subset=['pubdate']).copy()

if not pubdates_present_df.empty:
    # Ensure the pubdate column is in datetime format for min/max and year extraction
    pubdates_present_df['pubdate'] = pd.to_datetime(pubdates_present_df['pubdate'], errors='coerce')
    # Drop any rows that became NaT after this conversion (shouldn't happen if fillna('Unknown') was used)
    pubdates_present_df.dropna(subset=['pubdate'], inplace=True)


    if not pubdates_present_df.empty:
        # Display the earliest and latest publication dates
        earliest_date = pubdates_present_df['pubdate'].min()
        latest_date = pubdates_present_df['pubdate'].max()
        print(f"Earliest Publication Date: {earliest_date.strftime('%Y-%m-%d')}")
        print(f"Latest Publication Date: {latest_date.strftime('%Y-%m-%d')}")

        # Extract the year and count works per year
        pubdates_present_df['pub_year'] = pubdates_present_df['pubdate'].dt.year
        works_per_year = pubdates_present_df['pub_year'].value_counts().sort_index()

        print("\nNumber of works published per year:")
        display(works_per_year.head())
        display(works_per_year.tail())
    else:
        print("No works with valid publication dates to analyze after datetime conversion.")

else:
    print("No works with valid publication dates to analyze.")

In [ ]:
# Explore text data - Titles and Authors

print("Number of unique titles:", augmented_works_df['title'].nunique())
print("Number of unique authors:", augmented_works_df['author'].nunique())

print("\nExample Titles:")
display(augmented_works_df['title'].head())

print("\nExample Authors:")
display(augmented_works_df['author'].head())

In [ ]:
# Update the augmented data CSV file
out_path = DERIVED / "kenyan_works_augmented.csv"
augmented_works_df.to_csv(out_path, index=False)

print(f"Updated data saved to {out_path}")

In [ ]:
# Fill missing pubdates with a placeholder

# Convert the column to object type to allow string placeholder
augmented_works_df['pubdate'] = augmented_works_df['pubdate'].astype(object)

# Fill NaT values with 'Unknown'
augmented_works_df['pubdate'] = augmented_works_df['pubdate'].fillna('Unknown')

print("DataFrame Info after filling missing pubdates:")
augmented_works_df.info()

print("\nNumber of missing pubdates after filling:")
print(augmented_works_df['pubdate'].isna().sum())

display(augmented_works_df.head())

In [ ]:
# 3. Prepare for recommender system - Handle pubdate

# Convert 'pubdate' to datetime, coercing errors
augmented_works_df['pubdate'] = pd.to_datetime(augmented_works_df['pubdate'], errors='coerce')

print("DataFrame Info after pubdate conversion:")
augmented_works_df.info()

print("\nNumber of missing pubdates:")
print(augmented_works_df['pubdate'].isna().sum())

display(augmented_works_df.head())

In [ ]:
import re
import pandas as pd

# Curated Kenyan authors and notable works
kenyan_authors = {
    "Ngũgĩ wa Thiong’o": [
        "Weep Not, Child",
        "The River Between",
        "A Grain of Wheat",
        "Petals of Blood",
        "I Will Marry When I Want",
        "Wizard of the Crow"
    ],
    "Grace Ogot": [
        "The Promised Land",
        "Land Without Thunder",
        "A Call at Midnight",
        "The Other Woman"
    ],
    "Francis Imbuga": [
        "Betrayal in the City",
        "Aminata",
        "The Green Cross of Kafira"
    ],
    "Binyavanga Wainaina": [
        "One Day I Will Write About This Place",
        "How to Write About Africa"
    ],
    "Yvonne Adhiambo Owuor": [
        "Dust",
        "The Dragonfly Sea"
    ],
    "Marjorie Oludhe Macgoye": [
        "Coming to Birth",
        "The Present Moment"
    ],
    "Meja Mwangi": [
        "The Cockroach Dance",
        "Going Down River Road",
        "Kill Me Quick"
    ],
    "Muthoni Garland": [
        "Tracking the Scent of My Mother",
        "Odour of Fate",
        "Helicopter Beetles"
    ],
    "Mũkoma wa Ngũgĩ": [
        "Nairobi Heat",
        "Black Star Nairobi",
        "Hurling Words at Consciousness"
    ],
    "Rebecca Nandwa": [],
    "Rocha Chimera": [],
    "Okwiri Oduor": [],
    "Troy Onyango": []
}

# Normalize helper (assuming this is defined elsewhere, or redefine if necessary)
# def norm_key(s: str) -> str:
#     s = unidecode((s or "").lower())
#     s = re.sub(r"[\.,'’`\"\-]+", " ", s)
#     return re.sub(r"\s+", " ", s).strip()


# Use augmented_works_df instead of item_map
df = augmented_works_df.copy() # Use the main DataFrame
df["norm_title"] = df["title"].apply(norm_key)
df["norm_author"] = df["author"].apply(norm_key)

# Check presence
results = []
for author, works in kenyan_authors.items():
    norm_author = norm_key(author)
    author_present = df[df["norm_author"] == norm_author]
    if works:
        for work in works:
            norm_work = norm_key(work)
            # Check for match using normalized title and author
            work_present = author_present[author_present["norm_title"] == norm_work]
            results.append({
                "Author": author,
                "Work": work,
                "Present": not work_present.empty
            })
    else:
        # If no specific works listed, just check if the author is present
        results.append({
            "Author": author,
            "Work": "(any)",
            "Present": not author_present.empty
        })

summary_df = pd.DataFrame(results)
display(summary_df)

In [ ]:
# If summary_df isn't in memory, rebuild it quickly (safe fallback)
try:
    summary_df
except NameError:
    import re, pandas as pd
    def normalize_text(s):
        if not isinstance(s, str): return ""
        s = s.lower()
        s = re.sub(r"[^\w\s]", "", s)
        return s.strip()

    curated = {
        "Ngũgĩ wa Thiong’o": ["Weep Not, Child","The River Between","A Grain of Wheat","Petals of Blood","I Will Marry When I Want","Wizard of the Crow"],
        "Grace Ogot": ["The Promised Land","Land Without Thunder","A Call at Midnight","The Other Woman"],
        "Francis Imbuga": ["Betrayal in the City","Aminata","The Green Cross of Kafira"],
        "Binyavanga Wainaina": ["One Day I Will Write About This Place","How to Write About Africa"],
        "Yvonne Adhiambo Owuor": ["Dust","The Dragonfly Sea"],
        "Marjorie Oludhe Macgoye": ["Coming to Birth","The Present Moment"],
        "Meja Mwangi": ["The Cockroach Dance","Going Down River Road","Kill Me Quick"],
        "Muthoni Garland": ["Tracking the Scent of My Mother","Odour of Fate","Helicopter Beetles"],
        "Mũkoma wa Ngũgĩ": ["Nairobi Heat","Black Star Nairobi","Hurling Words at Consciousness"],
        "Rebecca Nandwa": [],
        "Rocha Chimera": [],
        "Okwiri Oduor": [],
        "Troy Onyango": []
    }

    df = item_map.copy()
    df["norm_title"]  = df["title"].apply(normalize_text)
    df["norm_author"] = df["author"].apply(normalize_text)
    rows = []
    for a, works in curated.items():
        na = normalize_text(a)
        sel = df[df["norm_author"]==na]
        if works:
            for w in works:
                rows.append({"Author": a, "Work": w,
                             "Present": not sel[sel["norm_title"]==normalize_text(w)].empty})
        else:
            rows.append({"Author": a, "Work": "(any)", "Present": not sel.empty})
    summary_df = pd.DataFrame(rows)

# Now extract missing, excluding "(any)"
missing_df = summary_df[(summary_df["Present"] == False) & (summary_df["Work"] != "(any)")].reset_index(drop=True)
missing_df

In [ ]:
import requests, time, re
import pandas as pd
from urllib.parse import urlencode
import logging # Import logging

log = logging.getLogger("somarec") # Get the logger instance

OL_SEARCH_URL = "https://openlibrary.org/search.json"
HEADERS = {"User-Agent": "SomaRec/0.9 (academic use)"}

def _norm(s: str) -> str:
    s = (s or "").lower()
    s = re.sub(r"[^\w\s]", " ", s)
    return re.sub(r"\s+", " ", s).strip()

def ol_search_title_author(title: str, author: str, limit: int = 10, delay: float = 0.2):
    params = {"title": title, "author": author, "limit": limit}
    r = requests.get(OL_SEARCH_URL, params=params, headers=HEADERS, timeout=45)
    r.raise_for_status()
    time.sleep(delay)
    js = r.json() or {}
    docs = js.get("docs", [])

    log.info(f"OL search for '{title}' by '{author}' returned {len(docs)} docs.") # Log initial count

    # Fetch descriptions for each doc using the Open Library API (details=true)
    # We'll do this for the top few results to avoid excessive requests
    enriched_count = 0
    for doc in docs[:5]: # Limit description fetching to top 5 results
        if doc.get('key'):
            try:
                details_url = f"https://openlibrary.org{doc['key']}.json"
                log.info(f"Attempting to fetch details for key: {doc['key']} from {details_url}") # Log the URL
                details_r = requests.get(details_url, headers=HEADERS, timeout=45)
                details_r.raise_for_status()
                details_js = details_r.json()
                log.info(f"Successfully fetched details for key: {doc['key']}. Response keys: {details_js.keys()}") # Log response keys
                description = details_js.get('description')
                if description:
                    # Description can be a string or a dict with a value key
                    if isinstance(description, dict) and 'value' in description:
                        doc['description'] = description['value']
                        log.info(f"Found dict description for key: {doc['key']}") # Log finding dict description
                    elif isinstance(description, str):
                        doc['description'] = description
                        log.info(f"Found string description for key: {doc['key']}") # Log finding string description
                    enriched_count += 1
                else:
                     log.info(f"No description found in details for key: {doc['key']}") # Log if no description found
            except Exception as e:
                log.warning(f"Failed to fetch OL details for {doc.get('key')}: {e}")

    log.info(f"Successfully enriched descriptions for {enriched_count} OL docs.") # Log enriched count
    return docs

def pick_best_ol(doc_list, title, author):
    """Pick best by strict title match + author token overlap, else fall back to token overlap score."""
    if not doc_list: return None
    nt = _norm(title); na = set(_norm(author).split())
    best, best_score = None, -1
    for d in doc_list:
        dt = _norm(d.get("title") or d.get("title_suggest") or "")
        anames = d.get("author_name") or []
        a_join = _norm(" ".join(anames))
        a_tokens = set(a_join.split())
        # exact title match bonus
        score = 2.0 if dt == nt else 0.0
        # token overlaps (title + author)
        score += len(set(nt.split()) & set(dt.split()))
        score += 0.5 * len(na & a_tokens)
        if score > best_score:
            best, best_score = d, score
    return best

def map_ol_to_row(d, source="ol_enrichment"):
    # fields may be absent; guard carefully
    title = d.get("title") or d.get("title_suggest")
    authors = d.get("author_name") or []
    publisher_list = d.get("publisher") or []
    year = d.get("first_publish_year")
    cover_i = d.get("cover_i")
    isbns = d.get("isbn") or []
    isbn13 = next((x for x in isbns if isinstance(x, str) and len(re.sub(r"\\D","", x))==13), None)
    isbn10 = next((x for x in isbns if isinstance(x, str) and len(re.sub(r"\\D","", x))==10), None)
    key = d.get("key")  # often /works/OL...W
    description = d.get("description") # Add description field

    row = {
        "title": title,
        "author": authors[0] if authors else None,
        "publisher": "\n".join(publisher_list[:5]) if publisher_list else None,
        "isbn13": isbn13, "isbn10": isbn10,
        "language": None,  # OL search not reliable here
        "pubdate": year,   # we'll cast to year later
        "ol_work_key": key,
        "cover_i": cover_i,
        "image_url": f"https://covers.openlibrary.org/b/id/{cover_i}-L.jpg" if cover_i else None,
        "source": source,
        "description": description # Include description in the row
    }
    return row

In [ ]:
enriched_rows = []
failures = []

for _, r in missing_df.iterrows():
    a, w = r["Author"], r["Work"]
    try:
        docs = ol_search_title_author(w, a, limit=10)
        pick = pick_best_ol(docs, w, a)
        if pick:
            enriched_rows.append(map_ol_to_row(pick, source="ol_enrichment"))
        else:
            failures.append((a, w, "no_match"))
    except Exception as e:
        failures.append((a, w, str(e)))

enriched_df = pd.DataFrame(enriched_rows)
print("Enriched rows:", len(enriched_df))
print("Failures (first 5):", failures[:5])
display(enriched_df.head())

In [ ]:
import pandas as pd
import numpy as np
import re

# 1) Load your pipeline output fresh from disk
BASE_PATH = "somarec_data/derived/kenyan_works_augmented.csv"
base_disk = pd.read_csv(BASE_PATH)

print("Loaded from disk:", base_disk.shape)
base_disk.head(3)

In [ ]:
# Ensure expected columns exist
REQ_COLS = ["title","author","publisher","language","image_url","isbn10","isbn13",
            "ol_work_key","cover_i","source","pubdate"]

def _ensure_cols(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    for c in REQ_COLS:
        if c not in df.columns:
            df[c] = None
    return df

# Start from the baseline on disk (not from the 1-row memory)
base = _ensure_cols(base_disk)

# Union with enrichments if they exist
to_concat = [base]

if "enriched_df" in globals() and isinstance(enriched_df, pd.DataFrame) and not enriched_df.empty:
    to_concat.append(_ensure_cols(enriched_df))
if "fallback_df" in globals() and isinstance(fallback_df, pd.DataFrame) and not fallback_df.empty:
    to_concat.append(_ensure_cols(fallback_df))

# In case you manually added items earlier (rare): plug them here if you kept them
if "manual_add_df" in globals() and isinstance(manual_add_df, pd.DataFrame) and not manual_add_df.empty:
    to_concat.append(_ensure_cols(manual_add_df))

# Concatenate while avoiding the FutureWarning by dropping all-NA non-REQ columns
cleaned = []
for d in to_concat:
    keep_cols = [c for c in d.columns if (c in REQ_COLS) or (~d[c].isna()).any()]
    cleaned.append(d.loc[:, keep_cols])

aw_raw = pd.concat(cleaned, ignore_index=True)

print("After union shape:", aw_raw.shape)
aw_raw.head(3)

In [ ]:
# If not installed already
# %pip install unidecode --quiet

import re, numpy as np, pandas as pd
from unidecode import unidecode

def norm_ascii(s: str) -> str:
    """Lowercase, ASCII fold, keep only [a-z0-9 ] and collapse spaces."""
    s = unidecode((s or "").strip().lower())
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

In [ ]:
# We assume you still have `aw_raw` (union of the 500-row CSV + enrichments).
# If not, re-run your "Loaded from disk" + "After union" cells first.

aw_fix = aw_raw.copy()

# Build robust normalized keys
aw_fix["title_norm"]  = aw_fix["title"].astype(str).apply(norm_ascii)
aw_fix["author_norm"] = aw_fix["author"].astype(str).apply(norm_ascii)

# Drop rows that have no usable title or author after normalization
aw_fix = aw_fix[(aw_fix["title_norm"] != "") & (aw_fix["author_norm"] != "")].copy()

# Safe de-dup: only if BOTH normalized title & author match
before = aw_fix.shape[0]
aw_fix = aw_fix.drop_duplicates(subset=["title_norm","author_norm"], keep="first")
after = aw_fix.shape[0]
print(f"Rows before dedup: {before} → after: {after}")

# tz-safe pubdate → year
def coerce_pubdate_naive(x):
    if pd.isna(x): return pd.NaT
    # year ints
    if isinstance(x, (int, np.integer)) and 0 < int(x) < 2100:
        return pd.Timestamp(int(x), 1, 1)
    # year floats
    if isinstance(x, (float, np.floating)):
        if not np.isnan(x) and 0 < int(x) < 2100:
            return pd.Timestamp(int(x), 1, 1)
        return pd.NaT
    # general string or dt
    t = pd.to_datetime(x, errors="coerce", utc=True)
    if pd.isna(t): return pd.NaT
    return pd.Timestamp(t).tz_localize(None)

aw_fix["pubdate"] = aw_fix["pubdate"].apply(coerce_pubdate_naive)
aw_fix["year"] = aw_fix["pubdate"].dt.year

# Year buckets & text channel for embeddings
bins   = [-np.inf, 1960, 1980, 1995, 2005, 2015, 2020, np.inf]
labels = ["<1960","60-79","80-94","95-04","05-14","15-19","2020+"]
aw_fix["_year_bucket"] = pd.cut(aw_fix["year"], bins=bins, labels=labels)

aw_fix["_text"] = (
    aw_fix["title"].fillna("") + " \n" +
    aw_fix["author"].fillna("") + " \n" +
    aw_fix["publisher"].fillna("")
).astype(str)

# Finalize canonical DF & item_map
aw = aw_fix.reset_index(drop=True).copy()
aw["item_id"] = np.arange(len(aw))
item_map = aw[["item_id","title","author","publisher","year","image_url"]].copy()

print("Canonicalized aw shape (robust):", aw.shape)
item_map.head(5)

In [ ]:
# %pip install sentence-transformers --quiet

from sentence_transformers import SentenceTransformer
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import OneHotEncoder, normalize
from sklearn.neighbors import NearestNeighbors

# Load or reuse ST model
if "st" not in globals() or st is None:
    st = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# Text embeddings
emb = st.encode(aw["_text"].tolist(), show_progress_bar=True, convert_to_numpy=True, normalize_embeddings=True)
X_text = csr_matrix(emb)

# One-hot metadata (light weight)
ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=True)
X_meta = ohe.fit_transform(aw[["author","publisher","_year_bucket"]])

meta_scale = (1.5 + 1.0 + 0.2) / 3.0
X_meta = X_meta * meta_scale

# Hybrid & normalize
X_hybrid = normalize(hstack([X_text, X_meta], format="csr"))

# KNN index
nn = NearestNeighbors(n_neighbors=min(50, max(2, X_hybrid.shape[0]-1)), metric="cosine", algorithm="brute")
nn.fit(X_hybrid)

print("Embeddings:", X_text.shape, "Hybrid:", X_hybrid.shape, "Items:", X_hybrid.shape[0])

In [ ]:
def norm_match(s):
    return norm_ascii(s)

df_check = item_map.copy()
df_check["title_norm"]  = df_check["title"].apply(norm_match)
df_check["author_norm"] = df_check["author"].apply(norm_match)

to_check = [
    ("Grace Ogot", "The Promised Land"),
    ("Grace Ogot", "A Call at Midnight"),
    ("Grace Ogot", "The Other Woman"),
    ("Francis Imbuga", "Betrayal in the City"),
    ("Francis Imbuga", "Aminata"),
    ("Francis Imbuga", "The Green Cross of Kafira"),
    ("Mũkoma wa Ngũgĩ", "Hurling Words at Consciousness"),
    ("Muthoni Garland", "Tracking the Scent of My Mother"),
    ("Muthoni Garland", "Odour of Fate"),
    ("Muthoni Garland", "Helicopter Beetles"),
]

rows = []
for a, w in to_check:
    na, nw = norm_match(a), norm_match(w)
    present = not df_check[(df_check["author_norm"]==na) & (df_check["title_norm"]==nw)].empty
    rows.append({"Author": a, "Work": w, "Present_now": present})
pd.DataFrame(rows)

In [ ]:
import numpy as np
import pandas as pd
import re
from sklearn.neighbors import NearestNeighbors

def _norm_ascii(s: str) -> str:
    # Must match whatever you used to build title_norm/author_norm
    from unidecode import unidecode
    import re
    s = unidecode((s or "").strip().lower())
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def _kneighbors_from_row(idx: int, n=50):
    distances, indices = nn.kneighbors(X_hybrid[idx], n_neighbors=min(n, nn.n_neighbors))
    return indices, distances

def _format_rows(indices, distances, k=10):
    rows = []
    for rank, (j, d) in enumerate(zip(indices[0][:k], distances[0][:k]), 1):
        im = item_map.iloc[int(j)]
        rows.append({
            "rank": rank,
            "score": 1.0 - float(d),
            "item_id": int(im["item_id"]),
            "title": im["title"],
            "author": im["author"],
            "publisher": im["publisher"],
            "year": int(im["year"]) if not pd.isna(im["year"]) else None,
            "image_url": im["image_url"]
        })
    return pd.DataFrame(rows)

In [ ]:
def recommend_by_title_base(title_query: str, k: int = 10):
    # Find best base row by normalized title contains
    q = _norm_ascii(title_query)
    m = item_map[item_map["title"].apply(_norm_ascii).str.contains(re.escape(q), na=False)]
    if m.empty:
        # token overlap fallback
        tokens = set(q.split())
        scores = item_map["title"].map(lambda t: len(tokens & set(_norm_ascii(t).split()))/(len(tokens)+1e-9))
        base_idx = int(scores.idxmax())
    else:
        base_idx = int(m.index[0])

    idxs, dists = _kneighbors_from_row(base_idx, n=max(50, k+1))
    # drop self if present
    if len(idxs[0]) and idxs[0][0] == base_idx:
        idxs = np.array([idxs[0][1:]]); dists = np.array([dists[0][1:]])
    return _format_rows(idxs, dists, k=k)

def recommend_by_id_base(item_id: int, k: int = 10):
    row_idx = int(item_map.index[item_map["item_id"] == item_id][0])
    idxs, dists = _kneighbors_from_row(row_idx, n=max(50, k+1))
    if len(idxs[0]) and idxs[0][0] == row_idx:
        idxs = np.array([idxs[0][1:]]); dists = np.array([dists[0][1:]])
    return _format_rows(idxs, dists, k=k)

# If you want free-text query now:
# We’ll reuse your encoder: ST or TF-IDF (you likely use ST now)
def _encode_query(text: str):
    # Uses the same SentenceTransformer 'st' you built; if you’re on TF-IDF, swap to tfidf.transform([text])
    from scipy.sparse import csr_matrix
    from sklearn.preprocessing import normalize
    v = st.encode([text], convert_to_numpy=True, normalize_embeddings=True)
    return normalize(csr_matrix(v))

def recommend_for_query_base(text: str, k: int = 10):
    Q = _encode_query(text)
    distances, indices = nn.kneighbors(Q, n_neighbors=min(50, nn.n_neighbors))
    return _format_rows(indices, distances, k=k)

In [ ]:
# Fast author lookup
_row_author = item_map["author"].fillna("").tolist()
def _author_of_idx(idx: int) -> str:
    return _row_author[idx] if 0 <= idx < len(_row_author) else ""

def rerank_diversity_penalty(indices, distances, k=10, same_author_penalty=0.15):
    cand = list(zip(map(int, indices[0]), map(float, distances[0])))
    picked, used = [], set()
    while cand and len(picked) < k:
        best, best_score = None, -1e9
        for j, d in cand:
            a = _author_of_idx(j)
            base = 1.0 - d
            score = base - (same_author_penalty if a and a in used else 0.0)
            if score > best_score:
                best, best_score = (j, d), score
        picked.append(best)
        used.add(_author_of_idx(best[0]))
        cand = [(j, d) for (j, d) in cand if j != best[0]]
    p_idx = np.array([[j for j, _ in picked]]); p_dst = np.array([[d for _, d in picked]])
    return p_idx, p_dst

def rerank_author_cap(indices, distances, k=10, max_per_author=2):
    counts = {}
    out = []
    for j, d in zip(map(int, indices[0]), map(float, distances[0])):
        a = _author_of_idx(j)
        counts[a] = counts.get(a, 0) + 1
        if counts[a] <= max_per_author:
            out.append((j, d))
        if len(out) == k:
            break
    if len(out) < k:
        extra = list(zip(map(int, indices[0]), map(float, distances[0])))
        for j, d in extra:
            if (j, d) not in out:
                out.append((j, d))
            if len(out) == k:
                break
    p_idx = np.array([[j for j, _ in out]]); p_dst = np.array([[d for _, d in out]])
    return p_idx, p_dst

In [ ]:
def recommend_by_title_diverse(title_query: str, k=10, penalty=0.15, cap=None):
    # Get base idx
    q = _norm_ascii(title_query)
    m = item_map[item_map["title"].apply(_norm_ascii).str.contains(re.escape(q), na=False)]
    if m.empty:
        tokens = set(q.split())
        scores = item_map["title"].map(lambda t: len(tokens & set(_norm_ascii(t).split()))/(len(tokens)+1e-9))
        base_idx = int(scores.idxmax())
    else:
        base_idx = int(m.index[0])

    # KNN candidates
    distances, indices = nn.kneighbors(X_hybrid[base_idx], n_neighbors=min(200, nn.n_neighbors))
    # drop self
    if indices[0][0] == base_idx:
        indices = np.array([indices[0][1:]]); distances = np.array([distances[0][1:]])

    if cap is not None:
        p_idx, p_dst = rerank_author_cap(indices, distances, k=k, max_per_author=cap)
    else:
        p_idx, p_dst = rerank_diversity_penalty(indices, distances, k=k, same_author_penalty=penalty)

    return _format_rows(p_idx, p_dst, k=k)

def recommend_for_query_diverse(text: str, k=10, penalty=0.15, cap=None):
    Q = _encode_query(text)
    distances, indices = nn.kneighbors(Q, n_neighbors=min(200, nn.n_neighbors))
    if cap is not None:
        p_idx, p_dst = rerank_author_cap(indices, distances, k=k, max_per_author=cap)
    else:
        p_idx, p_dst = rerank_diversity_penalty(indices, distances, k=k, same_author_penalty=penalty)
    return _format_rows(p_idx, p_dst, k=k)

In [ ]:
def _apply_filters(candidate_indices, language=None, year_min=None, year_max=None, blacklist_authors=None):
    mask = []
    bl = set(a.lower() for a in (blacklist_authors or []))
    for j in candidate_indices:
        row = item_map.iloc[int(j)]
        ok = True
        # language (if present in aw)
        if language is not None and "language" in aw.columns:
            lang = aw.iloc[int(j)]["language"]
            ok &= (isinstance(lang, str) and lang.lower() == language.lower())
        # year range
        y = row["year"]
        if year_min is not None:
            ok &= (pd.isna(y) or y >= year_min)
        if year_max is not None:
            ok &= (pd.isna(y) or y <= year_max)
        # author blacklist
        a = (row["author"] or "").lower()
        if bl:
            ok &= (a not in bl)
        mask.append(ok)
    return np.array(mask, dtype=bool)

def recommend_by_id_filtered(item_id: int, k=10, language=None, year_min=None, year_max=None, blacklist_authors=None):
    row_idx = int(item_map.index[item_map["item_id"] == item_id][0])
    distances, indices = nn.kneighbors(X_hybrid[row_idx], n_neighbors=min(200, nn.n_neighbors))
    idxs = indices[0]; dists = distances[0]
    # drop self
    if len(idxs) and idxs[0] == row_idx:
        idxs = idxs[1:]; dists = dists[1:]
    mask = _apply_filters(idxs, language, year_min, year_max, blacklist_authors)
    idxs = idxs[mask][:k]; dists = dists[mask][:k]
    return _format_rows(np.array([idxs]), np.array([dists]), k=k)

In [ ]:
seed_row = item_map[item_map["title"].notna()].head(1)
if not seed_row.empty:
    seed_title = seed_row.iloc[0]["title"]
    seed_id    = int(seed_row.iloc[0]["item_id"])
    print("Seed:", seed_title)

    print("\nBaseline:")
    display(recommend_by_title_base(seed_title, k=10))

    print("\nDiversified (penalty=0.15):")
    display(recommend_by_title_diverse(seed_title, k=10, penalty=0.15))

    print("\nFiltered (>=1990, blacklist seed author):")
    display(recommend_by_id_filtered(seed_id, k=10, year_min=1990, blacklist_authors=[seed_row.iloc[0]["author"]]))
else:
    print("No seed row found.")

In [ ]:
# Comprehensive Kenyan literary & CBC fasihi themes dictionary
themes = {
    # 🧭 Cultural & Identity
    "Cultural heritage": ["heritage", "tradition", "identity", "urithi wa kitamaduni", "lugha", "utamaduni"],
    "Indigenous knowledge": ["maarifa ya jadi", "indigenous", "traditional wisdom", "folk knowledge"],
    "Language and identity": ["lugha", "identity", "utambulisho", "multilingualism"],
    "Urban vs rural life": ["mjini", "vijijini", "urbanization", "rural", "maisha ya mjini", "maisha ya vijijini"],
    "Traditional beliefs": ["imani za jadi", "superstition", "beliefs", "rituals"],
    "Generational conflict": ["migogoro ya vizazi", "generation gap", "elders vs youth"],
    "Naming and symbolism": ["majina", "symbolism", "ishara", "totems"],
    "Folklore and oral traditions": ["hadithi za jadi", "oral literature", "simulizi", "myths", "legends"],
    "Ethnic diversity": ["tofauti za kikabila", "ethnicity", "tribes", "diversity"],
    "Religion and spirituality": ["dini", "spirituality", "faith", "kiroho"],

    # ⚖️ Social & Political
    "Colonialism and resistance": ["ukoloni", "resistance", "struggle", "freedom", "uhuru"],
    "Post-independence struggles": ["changamoto baada ya uhuru", "nation building", "development"],
    "Corruption and governance": ["ufisadi", "governance", "leadership", "betrayal", "usaliti", "justice", "utawala"],
    "Gender roles and feminism": ["majuku ya kijinsia", "usawa", "feminism", "patriarchy"],
    "Class and inequality": ["tabaka", "inequality", "social class", "wealth gap"],
    "Youth and unemployment": ["vijana", "ukosefu wa ajira", "hope", "dreams", "peer pressure", "ambitions"],
    "Education and opportunity": ["elimu", "school", "teacher", "learning", "fursa", "thamani ya elimu"],
    "Political violence": ["vurugu za kisiasa", "violence", "elections", "conflict"],
    "Leadership and betrayal": ["uongozi", "usaliti", "leaders", "betrayal", "bad governance"],
    "Civic responsibility": ["wajibu wa raia", "citizenship", "responsibility"],

    # 💔 Emotional & Psychological
    "Love and heartbreak": ["mapenzi", "love", "heartbreak", "usuhuba", "ndoa", "usaliti"],
    "Loneliness and isolation": ["upweke", "isolation", "kutengwa"],
    "Trauma and healing": ["mshtuko", "trauma", "healing", "uponyaji"],
    "Hope and despair": ["tumaini", "hope", "despair", "kukata tamaa"],
    "Identity crisis": ["mgogoro wa utambulisho", "identity", "crisis"],
    "Mental health": ["afya ya akili", "psychology", "stress", "depression"],

    # 🌍 Environmental & Economic
    "Land and displacement": ["ardhi", "uhamisho", "environment", "migration", "displacement"],
    "Climate and nature": ["hali ya hewa", "climate", "nature", "environment"],
    "Agriculture and food security": ["kilimo", "chakula", "food security", "farming"],
    "Urbanization": ["uendelezaji wa miji", "urban", "city life"],
    "Resource exploitation": ["unyonyaji wa rasilimali", "resources", "exploitation", "mining"],
    "Migration and diaspora": ["uhamiaji", "diaspora", "migration", "mataifa ya nje"],
    "Environmental conservation": ["uhifadhi wa mazingira", "conservation", "sustainability"],

    # 📚 Literary & Philosophical
    "Storytelling as resistance": ["usimulizi", "resistance", "oral literature"],
    "Morality and ethics": ["maadili", "ethics", "tabia", "morality"],
    "Fate and free will": ["majaliwa", "hiari", "destiny", "choice"],
    "Truth and deception": ["ukweli", "uongo", "truth", "lies", "deception"],
    "Power of language": ["nguvu ya lugha", "language", "expression"],
    "Silence and voice": ["kimya", "sauti", "voice", "silence"],
    "Good vs evil": ["mema", "maovu", "good", "evil"],

    # 👥 Family & Community (CBC emphasis)
    "Family unity": ["umoja wa familia", "family", "unity"],
    "Conflict in families": ["migogoro ya kifamilia", "family conflict"],
    "Marriage and relationships": ["ndoa", "mahusiaono", "relationships"],
    "Parent-child relationships": ["mzazi", "mtoto", "parent", "child"],
    "Community solidarity": ["ushirikiano wa jamii", "community", "solidarity"],

    # 🎓 Education & Youth (CBC emphasis)
    "Ignorance": ["ujinga", "ignorance"],
    "Peer pressure": ["shinikizo la marafiki", "peer pressure"],
    "Dreams and ambitions": ["ndoto", "matarajio", "ambitions", "dreams"],

    # 🌍 Social & Moral (CBC emphasis)
    "Greed": ["ulafi", "greed"],
    "Poverty": ["umaskini", "poverty"],
    "Wealth and materialism": ["utajiri", "mali", "materialism"],
    "Generosity": ["ukarimu", "generosity"],
    "Selfishness": ["ubinafsi", "selfishness"],
    "Honesty vs dishonesty": ["ukweli", "uongo", "honesty", "dishonesty"],
    "Hard work": ["bidii", "hard work"],
    "Laziness": ["uvivu", "laziness"],

    # ⚖️ Political & Governance (CBC emphasis)
    "Bad governance": ["utawala mbaya", "bad governance"],
    "Justice and injustice": ["haki", "dhuluma", "justice", "injustice"],
    "Oppression": ["ukandamizaji", "oppression"],
    "Resistance and struggle": ["mapambano", "struggle", "resistance"],
}


In [ ]:
# ---- Step 1: Generate Theme Embeddings ----
# Ensure themes dictionary is available (defined in cell c1MCSCQFZHTv)
if 'themes' not in globals():
    print("Error: 'themes' dictionary not found. Please run the cell defining the themes.")
    theme_embeddings = {}
else:
    from sentence_transformers import SentenceTransformer
    import numpy as np
    from sklearn.metrics.pairwise import cosine_similarity

    # Load or reuse ST model
    # Assuming 'model' for theme embeddings is defined here or in a previous cell
    if 'model' not in globals():
         try:
              model = SentenceTransformer('all-MiniLM-L6-v2')
         except Exception as e:
              print(f"Error loading SentenceTransformer model: {e}")
              model = None


    if model:
        print("Generating theme embeddings...")
        theme_embeddings = {theme: model.encode(keywords) for theme, keywords in themes.items()}
        print("Theme embeddings generated.")
    else:
        print("SentenceTransformer model not loaded. Cannot generate theme embeddings.")
        theme_embeddings = {}


# ---- Step 2: Compute Similarities ----
def get_top_themes(text, top_n=3):
    if not theme_embeddings or not model:
        return [] # Return empty list if themes or model are not loaded
    try:
        text_vec = model.encode([text])
        similarities = {
            theme: np.mean(cosine_similarity(text_vec, np.array(embeds)).flatten())
            for theme, embeds in theme_embeddings.items()
        }
        return sorted(similarities.items(), key=lambda x: x[1], reverse=True)[:top_n]
    except Exception as e:
        print(f"Error computing theme similarities: {e}")
        return []


# ---- Step 3: Apply to Dataset ----
# Ensure the canonicalized DataFrame 'aw' is available (defined after canonicalization, e.g., cell 3MAiGBdUUu_R)
if 'aw' not in globals() or not isinstance(aw, pd.DataFrame) or aw.empty or '_text' not in aw.columns:
    print("Error: Canonicalized DataFrame 'aw' with '_text' column not found or is empty.")
    print("Please ensure the data loading, harmonization, deduplication, and canonicalization steps have run successfully.")
    # Create an empty DataFrame to prevent further errors
    aw = pd.DataFrame()
    can_apply_themes = False
else:
    print("Using existing canonicalized DataFrame 'aw'.")
    can_apply_themes = True


if can_apply_themes and theme_embeddings:
    print("Applying theme extraction to DataFrame...")
    # Apply theme extraction to the correct DataFrame 'aw' and column '_text'
    aw['themes'] = aw['_text'].apply(lambda x: [t[0] for t in get_top_themes(x)])
    print("Theme extraction complete.")
    print("\nDataFrame with themes:")
    display(aw[['title', 'author', 'themes']].head())
elif not can_apply_themes:
    print("Cannot apply theme extraction as the DataFrame 'aw' is not ready.")
elif not theme_embeddings:
     print("Theme embeddings not created. Cannot apply theme extraction.")



In [ ]:
import pandas as pd
import numpy as np

# Ensure the 'aw' DataFrame exists and has a 'themes' column
if 'aw' in globals() and isinstance(aw, pd.DataFrame) and 'themes' in aw.columns:
    # Extract all unique themes from the list in the 'themes' column
    unique_themes = set()
    for theme_list in aw['themes'].dropna():
        if isinstance(theme_list, list):
            unique_themes.update(theme_list)
        # Handle cases where themes might not be in a list (though the previous step should ensure lists)
        elif isinstance(theme_list, str):
             # Assuming comma-separated string if not a list, adjust if format is different
             unique_themes.update([t.strip() for t in theme_list.split(',') if t.strip()])

    # Convert the set of unique themes to a DataFrame
    unique_themes_df = pd.DataFrame(sorted(list(unique_themes)), columns=['Unique Themes'])

    # Define the output path
    output_csv_path_unique_themes = DERIVED / "unique_themes.csv"

    # Save the DataFrame to a CSV file
    unique_themes_df.to_csv(output_csv_path_unique_themes, index=False)

    log.info(f"Saved unique themes to {output_csv_path_unique_themes}")
    display(unique_themes_df.head())

else:
    log.error("DataFrame 'aw' not found or does not contain a 'themes' column. Cannot extract unique themes.")

In [ ]:
# Save the DataFrame with themes to a CSV
output_csv_path_with_themes = DERIVED / "kenyan_works_with_themes.csv"
aw.to_csv(output_csv_path_with_themes, index=False)
log.info(f"Saved DataFrame with themes to {output_csv_path_with_themes}")

# Create and save a CSV with just title, author, and themes
title_author_themes_df = aw[['title', 'author', 'themes']].copy()
output_csv_path_title_author_themes = DERIVED / "kenyan_works_title_author_themes.csv"
title_author_themes_df.to_csv(output_csv_path_title_author_themes, index=False)
log.info(f"Saved title, author, themes mapping to {output_csv_path_title_author_themes}")

In [ ]:
import pandas as pd
import ast

# Load the file you prepared
INPUT_PATH = DERIVED / "kenyan_works_title_author_themes.csv"  # Use the DERIVED path
OUTPUT_PATH = DERIVED / "book_themes_final_linkage.csv"    # Use the DERIVED path

try:
    df = pd.read_csv(INPUT_PATH)
except FileNotFoundError:
    print(f"Error: Could not find {INPUT_PATH}. Please check the file path.")
    # Exit or handle the error as appropriate for your workflow
    # For this example, we'll just print the error and stop execution
    raise # Re-raise the exception to stop the cell execution


# 1. Safely convert the 'themes' string column back into a Python list
# Handles NaN/missing values by turning them into an empty list
df['themes'] = df['themes'].apply(
    lambda x: ast.literal_eval(x) if pd.notna(x) and isinstance(x, str) and x.strip().startswith('[') else []
)

# 2. Explode the list column to create one row per book-theme association
book_theme_links = df.explode('themes').dropna(subset=['themes']).copy()

# 3. Rename the theme column for clarity and save the final relational file
book_themes_links_final = book_theme_links[['title', 'author', 'themes']].rename(
    columns={'themes': 'theme_name'}
)
book_themes_links_final.to_csv(OUTPUT_PATH, index=False)

print(f"✅ Generated {OUTPUT_PATH} with {len(book_themes_links_final)} book-theme assignments.")

# Optional: Display the head of the new dataframe
print("\nHead of the generated linkage file:")
display(book_themes_links_final.head())

In [ ]:
# ---- Step 4: Generate Book Embeddings ----
# Ensure the canonicalized DataFrame 'aw' is available (defined after canonicalization, e.g., cell 3MAiGBdUUu_R)
if 'aw' not in globals() or not isinstance(aw, pd.DataFrame) or aw.empty or '_text' not in aw.columns:
    print("Error: Canonicalized DataFrame 'aw' with '_text' column not found or is empty.")
    print("Please ensure the data loading, harmonization, deduplication, and canonicalization steps have run successfully.")
    # Cannot proceed without the necessary data
    book_embeddings = None
else:
    print("Using existing canonicalized DataFrame 'aw' for embeddings.")
    # Use the 'model' defined in the theme extraction cell (Ry-5tq2O4zoJ)
    if 'model' not in globals():
         print("Error: SentenceTransformer model 'model' not found. Please run the theme extraction cell first.")
         book_embeddings = None
    else:
        print("Generating book embeddings...")
        # Use the correct DataFrame 'aw' and the correct column '_text'
        book_embeddings = model.encode(aw['_text'].tolist())
        print("Book embeddings generated.")


# ---- Step 5: Compute Cosine Similarity for Recommendations ----
if book_embeddings is not None:
    from sklearn.metrics.pairwise import cosine_similarity
    cos_sim = cosine_similarity(book_embeddings)
    print("Cosine similarity matrix computed.")

    def recommend_books(book_title, top_n=5):
        if 'aw' not in globals() or aw is None or aw.empty:
             print("Error: 'aw' DataFrame not available for recommendations.")
             return pd.DataFrame()

        # Use 'aw' DataFrame for recommendations
        match = aw[aw['title'].str.lower() == book_title.lower()]
        if match.empty:
            print(f"Error: Book '{book_title}' not found in the dataset.")
            return pd.DataFrame()

        idx = match.index[0]
        sim_scores = list(enumerate(cos_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        # Skip the first element as it is the book itself
        top_books_indices = [i[0] for i in sim_scores[1:top_n+1]]

        # Return relevant columns from the 'aw' DataFrame
        return aw.iloc[top_books_indices][['title', 'author', 'themes']]
else:
    print("Book embeddings not generated. Cannot compute cosine similarity or provide recommendations.")

In [ ]:
recommend_books("Bad governance")

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

pca = PCA(n_components=2)
reduced = pca.fit_transform(book_embeddings)

plt.figure(figsize=(10,6))
plt.scatter(reduced[:,0], reduced[:,1], alpha=0.6)
plt.title("Book Embeddings Distribution")
plt.show()


**Reasoning**:
Define the `recommend_by_theme` function to filter the DataFrame based on the provided theme and return the top k results with relevant columns.



In [ ]:
!pip install rapidfuzz --quiet
import pandas as pd
from rapidfuzz import process
import re
from unidecode import unidecode

# Assume 'themes' dictionary is globally available from a previous cell
# Assume 'aw' DataFrame is globally available from a previous cell

def norm_ascii(s: str) -> str:
    """Lowercase, ASCII fold, keep only [a-z0-9 ] and collapse spaces."""
    s = unidecode((s or "").strip().lower())
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

# Pre-normalize theme keywords once when themes dictionary is defined or loaded
# For now, we'll do it here assuming 'themes' is already loaded.
# In a more robust pipeline, this would be done immediately after defining 'themes'.
normalized_themes_keywords = {}
if 'themes' in globals():
    for theme, keywords in themes.items():
        normalized_themes_keywords[theme] = [norm_ascii(kw) for kw in keywords]
else:
    print("Warning: 'themes' dictionary not found. Theme recommendations may not work correctly.")


def recommend_by_theme(theme_query: str, k: int = 10) -> pd.DataFrame:
    """
    Recommends books based on a given theme query, using fuzzy matching against normalized keywords.
    """
    if 'aw' not in globals() or not isinstance(aw, pd.DataFrame) or aw.empty or 'themes' not in aw.columns:
        print("Error: Canonicalized DataFrame 'aw' with 'themes' column not found or is empty.")
        return pd.DataFrame()

    if not normalized_themes_keywords:
         print("Error: Normalized theme keywords not available. Cannot perform theme recommendations.")
         return pd.DataFrame()

    # Normalize the input query
    normalized_query = norm_ascii(theme_query)

    # Build a flat list of all normalized keywords with their corresponding theme name
    all_normalized_keywords_flat = []
    for theme_name, keywords in normalized_themes_keywords.items():
        for keyword in keywords:
            if keyword: # Avoid empty strings
                all_normalized_keywords_flat.append((keyword, theme_name))

    if not all_normalized_keywords_flat:
        print("No normalized theme keywords available for matching.")
        return pd.DataFrame()


    # Fuzzy match the normalized input query against the flat list of normalized keywords
    # We need to use extractOne on the list of keywords, and then map back to the theme name
    # Let's find the best matching keyword first
    best_keyword_match = process.extractOne(
        normalized_query,
        [item[0] for item in all_normalized_keywords_flat], # List of keywords
        score_cutoff=60 # Adjust cutoff as needed
    )

    best_match_theme_name = None
    if best_keyword_match:
        matched_keyword = best_keyword_match[0]
        # Find the theme name associated with the matched keyword
        # This assumes each keyword uniquely maps to a theme in this context, which might not be strictly true
        # A more robust approach would involve scoring themes based on all their keywords
        # For simplicity, we'll find the first theme associated with the matched keyword
        for keyword, theme_name in all_normalized_keywords_flat:
            if keyword == matched_keyword:
                best_match_theme_name = theme_name
                break # Found the theme, exit loop

        print(f"Fuzzy matched '{theme_query}' to theme keyword '{matched_keyword}' (Theme: '{best_match_theme_name}').")

    if best_match_theme_name:
        # Filter rows where the matched theme is present in the 'themes' list
        def has_matched_theme(theme_list):
            if isinstance(theme_list, list):
                # Ensure themes in the list are treated as strings and compared in a case-insensitive manner
                return any(str(t).lower() == best_match_theme_name.lower() for t in theme_list)
            # Handle cases where the 'themes' column might not be a list (shouldn't happen if previous steps worked)
            return str(theme_list).lower() == best_match_theme_name.lower()


        filtered_df = aw[aw['themes'].apply(has_matched_theme)].copy() # Use .copy() to avoid SettingWithCopyWarning

        if filtered_df.empty:
            print(f"No books found for the theme: '{best_match_theme_name}'")
            return pd.DataFrame()

        return filtered_df[['title', 'author', 'themes', 'image_url']].head(k)

    else:
        print(f"No close matches found for '{theme_query}' among theme keywords.")
        # Optional: list some available themes
        # print(f"Available themes include: {list(themes.keys())[:10]} ...")
        return pd.DataFrame()

**Reasoning**:
The `recommend_by_theme` function has been defined. Now, test the function with an example theme query to ensure it works as expected and displays the results.



In [ ]:
# Test the recommend_by_theme function
example_theme = "love"
recommended_books_by_theme = recommend_by_theme(example_theme, k=5)

print(f"\nRecommended books for theme '{example_theme}':")
display(recommended_books_by_theme)

## Create a book visualization function

### Subtask:
Develop a function that takes a list of book items and displays their covers and relevant information in a grid format.


**Reasoning**:
Define the `display_books_grid` function to visualize book covers and information in a grid format, handling image loading and display.



In [ ]:
import math
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import numpy as np

def display_books_grid(df, k=10, cols=5):
    """
    Display up to k books from the DataFrame in a grid with covers and titles.
    Expects df to have at least 'title' and optionally 'image_url' columns.
    """
    # Select the books to display
    books_to_display = df.head(k)
    num_books = len(books_to_display)

    # Compute grid size
    rows = math.ceil(num_books / cols)

    fig, axes = plt.subplots(rows, cols, figsize=(cols * 3, rows * 4))

    # Normalize axes into a flat list
    if isinstance(axes, np.ndarray):
        axes = axes.flatten()
    else:
        axes = [axes]

    for i, (idx, book) in enumerate(books_to_display.iterrows()):
        ax = axes[i]
        ax.axis("off")

        # Try to display cover image if available
        img = None
        if "image_url" in book and pd.notnull(book["image_url"]):
            try:
                import requests
                from PIL import Image
                from io import BytesIO
                response = requests.get(book["image_url"], timeout=5)
                img = Image.open(BytesIO(response.content))
                ax.imshow(img)
            except Exception:
                ax.text(0.5, 0.5, "No Image", ha="center", va="center")

        # Add title text below image
        ax.set_title(str(book.get("title", "Untitled")), fontsize=9)

    # Hide any unused axes
    for j in range(i + 1, len(axes)):
        axes[j].axis("off")

    plt.tight_layout()
    plt.show()

print("`display_books_grid` function defined.")

**Reasoning**:
Test the `display_books_grid` function with the previously generated recommended books DataFrame to visualize the results.



In [ ]:
# Test the display function with the recommended books DataFrame
if 'recommended_books_by_theme' in globals() and not recommended_books_by_theme.empty:
    print("Displaying recommended books:")
    display_books_grid(recommended_books_by_theme, k=10)
else:
    print("No recommended books DataFrame found or it is empty. Cannot display.")

## Connect theme recommendations to visualization

### Subtask:
Use the theme-based recommendation function to get a list of books and then pass that list to the visualization function.


**Reasoning**:
Call the theme-based recommendation function and then the visualization function with the result.



In [ ]:
from IPython.display import display
# Call the recommend_by_theme function
selected_theme = "family, love"
recommended_books_for_theme = recommend_by_theme(selected_theme, k=10)

# Call the display_books_grid function with the recommended books
if not recommended_books_for_theme.empty:
    print(f"Displaying recommended books for theme: '{selected_theme}'")
    display_books_grid(recommended_books_for_theme, k=10)
else:
    print(f"No books found for theme: '{selected_theme}' to display.")

In [ ]:
# STEP 1: INSTALL & IMPORT DEPENDENCIES
!pip install faiss-cpu sentence-transformers pandas numpy matplotlib pillow requests tqdm


In [ ]:
import pandas as pd
import numpy as np
import faiss
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
from PIL import Image
import requests

In [ ]:
# STEP 2: LOAD YOUR DATASET
df = pd.read_csv("/content/somarec_data/derived/kenyan_works_augmented.csv")

# Inspect structure
print(df.head())

# Make sure it has the necessary columns
required_cols = ["Book-Title", "Book-Author", "Description", "Image-URL-L"]
for col in required_cols:
    if col not in df.columns:
        print(f"⚠️ Missing column: {col}")


In [ ]:
# STEP 3: COMBINE FIELDS INTO A SINGLE TEXT COLUMN
# Use the correct column names from kenyan_works_augmented.csv
df["combined_text"] = (
    df["title"].fillna('') + " by " +
    df["author"].fillna('') + ". " +
    df["description"].fillna('')
)

In [ ]:
# STEP 4: LOAD MODEL AND GENERATE EMBEDDINGS
model = SentenceTransformer("all-MiniLM-L6-v2")

# Generate embeddings
book_embeddings = model.encode(
    df["combined_text"].tolist(),
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
)

# Check shape
print("Embeddings shape:", book_embeddings.shape)


In [ ]:
# STEP 5: BUILD A FAISS INDEX FOR FAST SIMILARITY SEARCH
embedding_dim = book_embeddings.shape[1]
index = faiss.IndexFlatIP(embedding_dim)   # Inner product = cosine similarity (since normalized)
index.add(book_embeddings)

print(f"✅ Added {index.ntotal} book embeddings to FAISS index")


In [ ]:
# STEP 6: DEFINE SEARCH FUNCTION
def recommend_books(query, top_k=5):
    query_vec = model.encode([query], normalize_embeddings=True)
    D, I = index.search(query_vec, top_k)

    results = df.iloc[I[0]].copy()
    results["similarity_score"] = D[0]
    return results[["title", "author", "similarity_score", "description", "image_url"]] # Corrected column name

In [ ]:
# STEP 7: TEST RECOMMENDATION SEARCH
examples = ["colonialism", "Kikuyu culture", "love and betrayal", "environment", "religion"]

for query in examples:
    print(f"\n🔎 Search: '{query}'\n")
    results = recommend_books(query, top_k=5)
    display(results[["title", "author", "similarity_score"]])


In [ ]:
# STEP 8: VISUALIZE THE RECOMMENDED BOOKS (like your earlier fig, ax setup)
def visualize_recommendations(query, top_k=5):
    results = recommend_books(query, top_k)
    fig, ax = plt.subplots(1, top_k, figsize=(17, 5))
    fig.suptitle(f"Recommendations for '{query}'", fontsize=20, color="mediumseagreen")

    for i in range(top_k):
        url = results.iloc[i]["image_url"] # Corrected column name
        try:
            img = Image.open(requests.get(url, stream=True).raw)
            ax[i].imshow(img)
        except:
            ax[i].text(0.5, 0.5, "No Image", ha="center", va="center", fontsize=14)
        ax[i].axis("off")
        # Assuming recommend_books returns 'title' and 'similarity_score'
        title = results.iloc[i]["title"]
        score = results.iloc[i]["similarity_score"]
        ax[i].set_title(f"{title[:20]}...\n({score:.2f})", fontsize=12, color="purple", y=-0.25)

    plt.show()

# Example visualization
visualize_recommendations("fasihi", top_k=5)

In [ ]:
# STEP 11: INSTALL AND IMPORT EVALUATION LIBRARIES
!pip install scikit-learn seaborn

import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import silhouette_score
from sklearn.manifold import TSNE

In [ ]:
# STEP 12: HELPER FUNCTIONS FOR EVALUATION

def recommendation_diversity(recommended_books, embeddings):
    """Compute diversity using pairwise cosine distances."""
    if len(recommended_books) < 2:
        return 0
    subset = embeddings[recommended_books]
    cosine_sim = cosine_similarity(subset)
    diversity = 1 - np.mean(cosine_sim[np.triu_indices(len(subset), 1)])
    return diversity


def visualize_clusters(embeddings, sample_df, n_samples=300):
    """Visualize semantic clusters of books with t-SNE."""
    sample_idx = np.random.choice(len(embeddings), size=min(n_samples, len(embeddings)), replace=False)
    subset_embeds = embeddings[sample_idx]

    # Explicitly create a copy to avoid SettingWithCopyWarning
    subset_df = sample_df.iloc[sample_idx].copy()

    tsne = TSNE(n_components=2, perplexity=30, random_state=42)
    reduced = tsne.fit_transform(subset_embeds)
    subset_df["x"] = reduced[:, 0]
    subset_df["y"] = reduced[:, 1]

    plt.figure(figsize=(10, 8))
    sns.scatterplot(
        data=subset_df,
        x="x",
        y="y",
        hue="author", # Use the correct column name 'author'
        legend=False,
        s=40,
        alpha=0.7
    )
    plt.title("t-SNE Visualization of Semantic Clusters")
    plt.show()


def evaluate_query(query, embeddings, top_k=5):
    """Evaluate how coherent and diverse the top results are."""
    results = recommend_books(query, top_k)
    indices = results.index.tolist()
    diversity_score = recommendation_diversity(indices, embeddings)
    return results, diversity_score

In [ ]:
# STEP 13: EVALUATE QUERIES AND COLLECT DIVERSITY SCORES

# Define example queries (using the ones from STEP 7 for consistency)
examples = ["colonialism", "Kikuyu culture", "love and betrayal", "environment", "religion"]

diversity_scores = []
for query in examples:
    results, diversity_score = evaluate_query(query, book_embeddings, top_k=5)
    diversity_scores.append((query, diversity_score))

# STEP 14: VISUALIZE QUERY DIVERSITY
div_df = pd.DataFrame(diversity_scores, columns=["Query", "Diversity"])
plt.figure(figsize=(8, 5))
sns.barplot(x="Diversity", y="Query", data=div_df, palette="mako")
plt.title("Recommendation Diversity per Query")
plt.xlabel("Diversity Score (Higher = More Varied)")
plt.show()

In [ ]:
# STEP 15: CLUSTERING QUALITY (Silhouette Score)
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np

# Assume book_embeddings and df are available from previous steps

# Choose a reasonable number of clusters.
# You can tune this by testing 5–30 and seeing which gives best silhouette.
n_clusters = 15

kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)

# Use the existing book_embeddings for clustering
if 'book_embeddings' in globals() and book_embeddings is not None:
    cluster_labels = kmeans.fit_predict(book_embeddings)

    # Use the existing df DataFrame
    if 'df' in globals() and isinstance(df, pd.DataFrame):
        df["cluster"] = cluster_labels

        # Evaluate clustering quality
        # Use a smaller subset for silhouette_score calculation if embeddings are very large
        subset_size = min(500, len(book_embeddings))
        subset_idx = np.random.choice(len(book_embeddings), size=subset_size, replace=False)
        subset_embeds = book_embeddings[subset_idx]
        subset_labels = cluster_labels[subset_idx]

        # Ensure there are at least two unique labels in the subset for silhouette_score
        if len(np.unique(subset_labels)) > 1:
             sil_score = silhouette_score(subset_embeds, subset_labels, metric="cosine")
             print(f"🧭 Silhouette Score (semantic cohesion): {sil_score:.4f}")

             # Visualize how many books ended up in each cluster
             cluster_counts = df["cluster"].value_counts().sort_index()
             print("\n📊 Books per cluster:\n", cluster_counts)
        else:
            print("Could not calculate Silhouette score: Subset does not contain at least two unique cluster labels.")
            print("Consider increasing subset_size or verifying clustering results.")

    else:
        print("Error: DataFrame 'df' not found or is not a pandas DataFrame. Cannot add cluster labels.")

else:
    print("Error: 'book_embeddings' not found or is None. Please ensure embeddings are generated.")

In [ ]:
# STEP 16: VISUALIZE CLUSTER DISTRIBUTION (PCA / t-SNE)
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Make sure df["cluster"] and book_embeddings exist
if "book_embeddings" in globals() and "df" in globals() and "cluster" in df.columns:
    print("✅ Starting dimensionality reduction for visualization...")

    # --- 1. PCA first (to speed up t-SNE)
    pca = PCA(n_components=50, random_state=42)
    reduced_pca = pca.fit_transform(book_embeddings)

    # --- 2. t-SNE for 2D visualization
    tsne = TSNE(
        n_components=2,
        perplexity=30,
        learning_rate="auto",
        init="random",
        random_state=42,
        n_iter=1000
    )
    reduced_tsne = tsne.fit_transform(reduced_pca)

    # Add the 2D coordinates to df
    df["tsne_x"] = reduced_tsne[:, 0]
    df["tsne_y"] = reduced_tsne[:, 1]

    # --- 3. Plot with Matplotlib
    plt.figure(figsize=(10, 7))
    scatter = plt.scatter(
        df["tsne_x"],
        df["tsne_y"],
        c=df["cluster"],
        cmap="tab20",
        s=40,
        alpha=0.8
    )
    plt.colorbar(scatter, label="Cluster ID")
    plt.title("📚 Semantic Clusters of Kenyan Books (t-SNE Projection)")
    plt.xlabel("t-SNE Dimension 1")
    plt.ylabel("t-SNE Dimension 2")
    plt.show()

else:
    print("⚠️ Required data not found. Ensure 'df' and 'book_embeddings' with cluster labels exist.")


In [ ]:
# STEP 17.1: FAISS Voronoi (IVF) Index Setup

import faiss
import numpy as np

# Dimension of embeddings
d = book_embeddings.shape[1]

# Step 1: Define the base quantizer (FlatIP because embeddings are normalized)
quantizer = faiss.IndexFlatIP(d)

# Step 2: Create the IVF (Voronoi partitioned) index
nlist = 25  # number of Voronoi cells (you can tune: 10–50)
index_ivf = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_INNER_PRODUCT)

# Step 3: Train the index — FAISS performs internal KMeans clustering here (unsupervised)
print("🔹 Training FAISS IVF index (this performs unsupervised clustering)...")
index_ivf.train(book_embeddings.astype('float32'))
print("✅ Training complete.")

# Step 4: Add all book embeddings to the index
index_ivf.add(book_embeddings.astype('float32'))
print(f"📚 Indexed {index_ivf.ntotal} books into {nlist} clusters.")


In [ ]:
# STEP 17.2: Control search depth
index_ivf.nprobe = 8  # how many clusters to probe per search (tune 5–20)
print(f"🔍 Using nprobe = {index_ivf.nprobe} for broader semantic matching.")


In [ ]:
from sentence_transformers import SentenceTransformer

# Assume model already loaded, e.g.:
# model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def recommend_books_v2(query: str, top_k: int = 5):
    """
    Uses FAISS IVF Voronoi clustering to find semantically similar books.
    """
    # Encode query to vector
    query_vec = model.encode([query], convert_to_numpy=True, normalize_embeddings=True).astype('float32')

    # Search in FAISS IVF index
    D, I = index_ivf.search(query_vec, top_k)

    # Fetch top results
    results = df.iloc[I[0]].copy()
    results["similarity_score"] = D[0]

    print(f"🔎 Query: '{query}'")
    print(f"📖 Top {top_k} semantic matches:\n")
    return results[["title", "author", "similarity_score"]]


In [ ]:
example_queries = [
    "Kikuyu traditions and beliefs",
    "post-colonial education",
    "environmental conservation",
    "love and betrayal",
    "religious conflict"
]

for q in example_queries:
    display(recommend_books_v2(q, top_k=5))
    print("-" * 100)


In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Reduce dimensions for visualization
pca = PCA(n_components=2)
reduced_embeds = pca.fit_transform(book_embeddings)

# Assign cluster labels directly from FAISS (using quantizer search)
_, faiss_clusters = quantizer.search(book_embeddings.astype('float32'), 1)

plt.figure(figsize=(10, 6))
plt.scatter(reduced_embeds[:, 0], reduced_embeds[:, 1], c=faiss_clusters.squeeze(), cmap='tab20', s=10)
plt.title("FAISS Voronoi (IVF) Clusters of Kenyan Literature Embeddings")
plt.xlabel("PCA Dimension 1")
plt.ylabel("PCA Dimension 2")
plt.show()


In [ ]:
from sklearn.metrics import silhouette_score

silhouette = silhouette_score(book_embeddings, faiss_clusters.squeeze(), metric="cosine")
print(f"🧭 Silhouette Score (cluster cohesion): {silhouette:.4f}")


In [ ]:
def visualize_recommendations_grid(query, top_k=10, cols=5):
    """
    Gets recommendations using recommend_books_v2 and visualizes them in a grid.
    """
    recommended_books_df = recommend_books_v2(query, top_k)

    # Ensure the DataFrame has the necessary columns for display_books_grid
    # recommend_books_v2 returns "title", "author", "similarity_score"
    # display_books_grid expects "title" and optionally "image_url"
    # We need to add "image_url" back to the DataFrame if it's not there
    if 'image_url' not in recommended_books_df.columns:
        # This might not be the most efficient, but ensures the column exists.
        # A better approach might be to include image_url in the recommend_books_v2 output.
        # For now, let's fetch from the original df based on index.
        original_indices = recommended_books_df.index # Get original indices from the search result
        recommended_books_df['image_url'] = df.loc[original_indices, 'image_url'].values


    print(f"\nVisualizing recommendations for query: '{query}'")
    display_books_grid(recommended_books_df, k=top_k, cols=cols)

In [ ]:
# Example of using the new visualization function
visualize_recommendations_grid("church", top_k=8, cols=4)